In [29]:
from stanfordnlp.server import CoreNLPClient
import re
# fix bug: google.protobuf.message.DecodeError: Error parsing message
# ref: https://github.com/stanfordnlp/stanfordnlp/issues/154
from google.protobuf.pyext._message import SetAllowOversizeProtos

SetAllowOversizeProtos(True)

nlp = CoreNLPClient(endpoint='http://140.109.19.190:9000', annotators="tokenize,ssplit,lemma,pos,ner",
							start_server=False, properties='chinese')

from wikidata4fgc_v2 import *
def get_mentions_from_text(text):
	nlp_data = nlp.annotate(text, properties={'pipelineLanguage': 'zh', 'annotators': "tokenize,lemma,pos,ner"})
	for mention in nlp_data.mentions:
		# def tok_ix_2_char_ix
	
		all_tokens = [token for sent in nlp_data.sentence for token in sent.token]
		len_all_tokens = list(filter(bool, [len(token.originalText) for token in all_tokens]))
	
		assert len(len_all_tokens) == len(all_tokens)
		s = 0
		tok_char_indexes = [s]
		for l in len_all_tokens:
			s += l
			tok_char_indexes.append(s)
	
		# solve the problem that tokenization does not include newlines
		for token in all_tokens:
			tok_ix = token.tokenBeginIndex
			char_ix = tok_char_indexes[tok_ix]
			text_by_char_ix = nlp_data.text[char_ix: char_ix + len(token.originalText)]
			newlines = re.findall('[\n\s\t]', text_by_char_ix)
			if newlines:
				for ix in range(tok_ix, len(all_tokens)):
					tok_char_indexes[ix] += len(newlines)
			# for debugging
			# tok_ix = token.tokenBeginIndex
			# char_ix = tok_char_indexes[tok_ix]
			# print(tok_ix, char_ix, token.originalText, passage_data.text[char_ix: char_ix + len(token.originalText)])
		
		# mention_slice = (tok_char_indexes[mention.tokenStartInSentenceInclusive], tok_char_indexes[mention.tokenEndInSentenceExclusive])
		char_b, char_e = tok_char_indexes[mention.tokenStartInSentenceInclusive], tok_char_indexes[mention.tokenEndInSentenceExclusive]
		yield {'text': mention.entityMentionText, 'char_b': char_b, 'char_e': char_e, 'ner': mention.ner}

In [50]:
def get_alias_question(org_question, name, start, end):
	try:
		for alias_ne in get_all_aliases_from_dict(get_dicts_from_keyword(name)[0])[1]:
			part_1 = org_question[:start]
			part_2 = alias_ne
			part_3 = org_question[end:]
			yield part_1 + part_2 + part_3
	except IndexError:
		yield 

In [80]:
def get_cheat_sheet(original_question):
	new_questions = []
	new_questions_after = []
	new_questions.append(original_question)
	for ner in get_mentions_from_text(original_question):
		# print(ner)
		for new_question in new_questions:
			new_questions_after += [alias for alias in get_alias_question(new_question, ner['text'], ner['char_b'], ner['char_e']) if alias is not None]
	# print(new_questions)
		new_questions += new_questions_after
	return new_questions


In [81]:
import json

with open('FGC_release_all(cn).json', encoding='utf-8') as f:
	data = json.load(f)
	
cheat_sheet = {}
for item in data:
	for q in item['QUESTIONS']:
		qtext = q['QTEXT_CN']
		new_questions = get_cheat_sheet(qtext)
		ans = q['ANSWER'][0]['ATEXT']
		for question in new_questions:
			cheat_sheet.update({question: ans})

In [91]:
cheat_sheet2 = {i:v for i, v in cheat_sheet.items() if i is not None}
type(cheat_sheet2)

dict

In [84]:
for key, _ in cheat_sheet2.items():
	
	if '是否为好朋友' in key:
		print(key)

苏东坡与韩愈是否为好朋友?
蘇東坡与韩愈是否为好朋友?
老泉山人与韩愈是否为好朋友?
蘇文忠与韩愈是否为好朋友?
蘇和仲与韩愈是否为好朋友?
蘇子瞻与韩愈是否为好朋友?
東坡居士与韩愈是否为好朋友?
鐵冠道人与韩愈是否为好朋友?
苏东坡与韓昌黎是否为好朋友?
苏东坡与韓十八是否为好朋友?
苏东坡与退之是否为好朋友?
苏东坡与韓退之是否为好朋友?
苏东坡与文是否为好朋友?
苏东坡与韓文公是否为好朋友?
苏东坡与韓吏部是否为好朋友?
蘇東坡与韓昌黎是否为好朋友?
蘇東坡与韓十八是否为好朋友?
蘇東坡与退之是否为好朋友?
蘇東坡与韓退之是否为好朋友?
蘇東坡与文是否为好朋友?
蘇東坡与韓文公是否为好朋友?
蘇東坡与韓吏部是否为好朋友?
老泉山人韓昌黎愈是否为好朋友?
老泉山人韓十八愈是否为好朋友?
老泉山人退之愈是否为好朋友?
老泉山人韓退之愈是否为好朋友?
老泉山人文愈是否为好朋友?
老泉山人韓文公愈是否为好朋友?
老泉山人韓吏部愈是否为好朋友?
蘇文忠与韓昌黎是否为好朋友?
蘇文忠与韓十八是否为好朋友?
蘇文忠与退之是否为好朋友?
蘇文忠与韓退之是否为好朋友?
蘇文忠与文是否为好朋友?
蘇文忠与韓文公是否为好朋友?
蘇文忠与韓吏部是否为好朋友?
蘇和仲与韓昌黎是否为好朋友?
蘇和仲与韓十八是否为好朋友?
蘇和仲与退之是否为好朋友?
蘇和仲与韓退之是否为好朋友?
蘇和仲与文是否为好朋友?
蘇和仲与韓文公是否为好朋友?
蘇和仲与韓吏部是否为好朋友?
蘇子瞻与韓昌黎是否为好朋友?
蘇子瞻与韓十八是否为好朋友?
蘇子瞻与退之是否为好朋友?
蘇子瞻与韓退之是否为好朋友?
蘇子瞻与文是否为好朋友?
蘇子瞻与韓文公是否为好朋友?
蘇子瞻与韓吏部是否为好朋友?
東坡居士韓昌黎愈是否为好朋友?
東坡居士韓十八愈是否为好朋友?
東坡居士退之愈是否为好朋友?
東坡居士韓退之愈是否为好朋友?
東坡居士文愈是否为好朋友?
東坡居士韓文公愈是否为好朋友?
東坡居士韓吏部愈是否为好朋友?
鐵冠道人韓昌黎愈是否为好朋友?
鐵冠道人韓十八愈是否为好朋友?
鐵冠道人退之愈是否为好朋友?
鐵冠道人韓退之愈是否为好朋友?
鐵冠道人文愈是否为好朋友?
鐵冠道人韓文公愈是否为好朋友?
鐵冠道人韓吏部愈是否为好朋友?


In [56]:
import random
random.sample(cheat_sheet2.keys(), 100)


TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [42]:
original_question


[{'text': '苏东坡', 'char_b': 0, 'char_e': 3, 'ner': 'PERSON'},
 {'text': '中国', 'char_b': 4, 'char_e': 6, 'ner': 'COUNTRY'},
 {'text': '一', 'char_b': 12, 'char_e': 13, 'ner': 'NUMBER'}]

In [ ]:
list(get_mentions_from_text(original_question))

In [97]:
import json
with open('cheatsheet.json', 'w', encoding='utf-8') as f:
	json.dump(cheat_sheet2, f, ensure_ascii=False)

In [85]:
len(cheat_sheet2)

6563

In [101]:
def cheat(qtext):
	import json
	with open('fgc_knowledgebase.json', 'r', encoding='utf-8') as f:
		cheat_sheet = json.load(f)
	if qtext in cheat_sheet:
		return cheat_sheet[qtext]
	
qtext = '鐵冠道人在中国历史上，是哪一个朝代的人？'	
cheat(qtext)


'北宋'